In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    TimeoutException,
    NoSuchElementException,
    ElementClickInterceptedException,
    WebDriverException
)
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
import time

def scrape_player_stats(player_url):
    options = webdriver.ChromeOptions()
    options.add_argument('--start-maximized')
    driver = webdriver.Chrome(options=options)

    try:
        driver.get(player_url)

        # === Paso 1: Extraer nombre del jugador ===
        try:
            name_elem = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.TAG_NAME, "h2"))
            )
            player_name = name_elem.text.strip()
            print("Nombre del jugador:", player_name)
        except Exception as e:
            print("❌ Error al extraer nombre del jugador:", e)
            player_name = None

        # === Paso 2: Extraer nombre del equipo ===
        try:
            team_elem = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located(
                    (By.XPATH,
                     "(//div[contains(@class,'flex-d_column') and contains(@class,'jc_center')]//span[contains(@class,'textStyle_body')])[1]")
                )
            )
            team_name = team_elem.text.strip()
            print("Equipo:", team_name)
        except Exception as e:
            print("❌ Error al extraer nombre del equipo:", e)
            team_name = None

        # === Paso 3: Extraer posición del jugador ===
        try:
            pos_elem = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located(
                    (By.CSS_SELECTOR, "div.Box.oWZdE .Text.beCNLk")
                )
            )
            position = pos_elem.text.strip()
            print("Posición:", position)
        except Exception as e:
            print("❌ Error al extraer posición:", e)
            position = None

        # === Paso 4: Hacer click en el último box que abre la modal ===
        try:
            triggers = WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.Box.gRFdPz"))
            )
            trigger = triggers[-1]
            driver.execute_script("arguments[0].scrollIntoView({block:'center'})", trigger)
            time.sleep(0.5)
            ActionChains(driver).move_to_element(trigger).click().perform()
            print("✅ Click en el último box de stats realizado.")
        except Exception as e:
            print("❌ Error al hacer click en el último trigger de modal:", e)

        # === Paso 5: Esperar modal, limpiar overlays y extraer estadísticas ===
        stats = {}
        try:
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "span.ov-x_clip"))
            )
            time.sleep(2)

            driver.execute_script("""
                document.querySelectorAll('[id^="div-gpt-ad"], .overlay, .banner').forEach(el => el.style.display='none');
            """)

            labels = driver.find_elements(By.CSS_SELECTOR, "span[class*='ov-x_clip']")
            for label in labels:
                if not label.is_displayed():
                    continue
                try:
                    value = driver.execute_script("return arguments[0].nextElementSibling", label)
                    text_value = value.text.strip() if value else ""
                except Exception:
                    continue

                ltxt = label.text.strip()
                if ltxt and text_value:
                    stats[ltxt] = text_value

            if stats:
                print("🏅 Estadísticas extraídas:")
                for k, v in stats.items():
                    print(f"  • {k}: {v}")
            else:
                print("⚠️ No se encontraron estadísticas visibles.")
        except Exception as e:
            print("❌ Error extrayendo estadísticas:", e)

        # === Construir DataFrame final ===
        data = [
            ("Nombre del jugador", player_name),
            ("Equipo", team_name),
            ("Posición", position)
        ] + list(stats.items())

        df = pd.DataFrame(data, columns=["Campo", "Valor"])
        return df

    finally:
        driver.quit()


# Ejemplo de uso:
if __name__ == "__main__":
    url = "https://www.sofascore.com/es/football/player/diego-rossi/831809"
    df_resultado = scrape_player_stats(url)
    print("\n📊 DataFrame final:")
    print(df_resultado)

Nombre del jugador: Diego Rossi
Equipo: Columbus Crew
Posición: F
✅ Click en el último box de stats realizado.
🏅 Estadísticas extraídas:
  • Minutos jugados: 62'
  • Goles: 1
  • Asistencias: 1
  • Tiros a puerta: 1
  • Tiros fuera: 2
  • Tiros bloqueados: 0
  • Regates intentados (completados): 1 (0)
  • Ocas. claras falladas: 1
  • Toques: 43
  • Pases precisos: 24/27 (89%)
  • Pases clave: 2
  • Centros (acertados): 3 (2)
  • Pases largos (acertados): 1 (1)
  • Ocas. claras creadas: 2
  • Duelos en el suelo (ganados): 6 (4)
  • Duelos aéreos (ganados): 0 (0)
  • Posesiones perdidas: 10
  • Faltas: 0
  • Recibió falta: 3
  • Despejes: 0
  • Intercepciones: 0
  • Tackles totales: 1
  • Regateado: 0

📊 DataFrame final:
                               Campo          Valor
0                 Nombre del jugador    Diego Rossi
1                             Equipo  Columbus Crew
2                           Posición              F
3                    Minutos jugados            62'
4          